In [39]:
#%run 001_setup_path.ipynb
%run 001_setup_path_local.ipynb

In [40]:
output_path_results

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [41]:
df_name = 'fd.csv' # 'fd.csv' 'df.csv' #
target = 'fd_taken'  # 'fd_taken'   'Survived'

In [42]:
algo = 'RF' # RF LR NB SV all

In [43]:
supress_warnings = True #False

In [44]:
output_path_results

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [45]:
import os
os.getcwd()

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [46]:
# Load all methods
%run 011_a_flow_preprocess_non_interactive_v_0_2.ipynb

In [47]:
import sys
sys.path.append(methods_path)
from setup_input_002 import *

target,validation_set_name,columns_to_drop,columns_to_convert,df_processed_final = setup_input_new(df_name,algo)

In [48]:
# # %run setup_input_002.ipynb

# sys.path.append(methods_path)
# from setup_input_002 import *

# df_name,target,validation_set_name,columns_to_drop,columns_to_convert,df_processed_final,algo = setup_input()

In [49]:
df_name,target,validation_set_name,columns_to_drop,columns_to_convert,df_processed_final,algo

('fd.csv',
 'fd_taken',
 'validation_set.csv',
 [],
 [],
 'df_processed_final.csv',
 'RF')

In [50]:
output_path_results

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [51]:
algo

'RF'

In [52]:
log_to_file_algo(algo,output_path_results)

In [53]:
# Define data and target
path = output_path_csvs

# Load data
df = read_file(df_processed_final,path) 

In [54]:
# test point
df.head(10)

cust_id  casa_balance  other_balance  fd_taken  age
0        1           100            100         1   20
1        2           200            200         1   21
2        3           300            300         0   22
3        1           150            150         1   21
4        2           250            250         1   22
5        3            50             50         0   23

In [55]:
target

'fd_taken'

In [56]:
X = df.drop(target,axis=1)

In [57]:
y = df[target]

In [58]:
X.head(10)

cust_id  casa_balance  other_balance  age
0        1           100            100   20
1        2           200            200   21
2        3           300            300   22
3        1           150            150   21
4        2           250            250   22
5        3            50             50   23

In [59]:
y.head(10)

0    1
1    1
2    0
3    1
4    1
5    0
Name: fd_taken, dtype: int64

In [60]:
threshold=0.545
X,to_drop = drop_correlated_columns(X,threshold)

In [61]:
# test point
X.head(10)

cust_id  casa_balance
0        1           100
1        2           200
2        3           300
3        1           150
4        2           250
5        3            50

In [62]:
import os
os.getcwd()

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\csvs'

In [63]:
# Split data into train and test
test_size=0.20
random_state=100
X_train,X_test,y_train,y_test = split(X,y,test_size,random_state)

In [64]:
# Train the model
X_train,y_train,model = train_model(X_train,X_test,y_train,y_test,X,y,algo,df)
if(supress_warnings != True):
    if(algo == 'RF'):
        feature_importance(X,model) # Causing warning in log

In [65]:
import os
os.getcwd()

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\csvs'

In [66]:
# Save model
save_model(models_path,model,algo)

if(algo == 'SV'):
    # Predict the results
    pred,conf_matrix,clas_report,acc_score= predict_results(model,X_test,y_test)
else:
    # Predict the results    
    pred,pred_proba,conf_matrix,clas_report,acc_score= predict_results(model,X_test,y_test)
    
# Confusion Matrix
print('Confusion Matrix : \n ',conf_matrix)
log_to_file('Confusion Matrix :',output_path_results)
log_to_file_result(conf_matrix,output_path_results)

# Classification Report
print(' \n Classification Report : \n ',clas_report)
log_to_file(' Classification Report :',output_path_results)
log_to_file_result(clas_report,output_path_results)

# Accuracy
print(' \n Accuracy : \n ',acc_score*100)
log_to_file(' Accuracy :',output_path_results)
log_to_file_result(acc_score*100,output_path_results)


Confusion Matrix : 
  [[0 1]
 [0 1]]
path:
 C:\Users\sunitha G\dm_offshore\projects\fd_propensity\output\results
 
 Classification Report : 
               precision    recall  f1-score   support

          0       0.00      0.00      0.00         1
          1       0.50      1.00      0.67         1

avg / total       0.25      0.50      0.33         2

path:
 C:\Users\sunitha G\dm_offshore\projects\fd_propensity\output\results
 
 Accuracy : 
  50.0
path:
 C:\Users\sunitha G\dm_offshore\projects\fd_propensity\output\results


C:\Users\sunitha G\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [67]:
if(supress_warnings != True):
    roc_curve_plot(y_test,pred) # # Causing warning in log

In [68]:
# def decile_analysis1(decile_df,algo):
    
#     # No.of 1's by total 0's and 1's
# #     print(decile_df[decile_df.y_true==1])
#     print(decile_df[decile_df.y_true==1].shape[0])
#     print(decile_df.shape[0])
    
#     base_response_rate = np.round(100*decile_df[decile_df.y_true==1].shape[0]/decile_df.shape[0],decimals=2)
#     print('base_response_rate : \n',base_response_rate)
#     if(algo != 'SV'):
#         decile_df.sort_values(by='y_prob',inplace = True,ascending = False)
#         decile_df.reset_index(inplace = True)
#     decile_df['decile'] = np.nan
#     d =int(np.ceil(decile_df.shape[0]/10))
#     start = 0
#     end = d
#     for i in range(10):
#         decile_df.loc[start:end,['decile']] = i+1
#         start = start +d
#         end = end +d
#     qq = pd.crosstab(decile_df['decile'],decile_df['y_true'])
#     qq.columns = ['zero','one']   
#     qq['min_prob'] = decile_df.groupby(by=['decile']).min()['y_prob']
#     qq['max_prob'] = decile_df.groupby(by=['decile']).max()['y_prob']
#     qq['count'] = decile_df.groupby(by=['decile']).count()['y_prob']
#     qq['gain'] = np.round(100*qq['one']/qq['one'].sum(),decimals=2)
#     qq['cum_gain'] = np.cumsum(qq['gain'])
#     qq['penetration'] = (qq['one']/qq['count'])*100  

# #     print('df:\n',decile_df['decile'])
# #     print('qq one:\n',qq['one'])
    
# #     print('both tab:\n',pd.crosstab(qq['min_prob'],qq['max_prob'],margins=True))
# #     print('minn_prob:\n', qq['min_prob'])
# #     print('qq:\n',qq)
# #     print('final:\n',pd.crosstab(qq,margins=True))
#     log_to_file('Decile results :')
#     log_to_file_result(qq)
    
#     return qq

In [69]:
decile_df = pd.DataFrame()
decile_df['y_true'] = y_test
# decile_df['y_prob'] = pred_proba
decile_df['y_pred'] = pred
# decile_df['decile'] = pd.cut(decile_df['y_prob'],10,labels=range(10,0,-1))
if(algo != 'SV'):
    decile_df['y_prob'] = pred_proba
    decile_df['decile'] = pd.cut(decile_df['y_prob'],10,labels=range(10,0,-1))


In [70]:
output_path_results

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [71]:
import os
os.getcwd()

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [72]:
decile_results = decile_analysis(decile_df,algo)
print(decile_results)
log_to_file_algo(decile_results,output_path_results)

base_response_rate : 
 50.0
        zero  one  min_prob  max_prob  count   gain  cum_gain  penetration
decile                                                                    
1          0    1      0.99      0.99      1  100.0     100.0        100.0
2          1    0      0.65      0.65      1    0.0     100.0          0.0


In [73]:
import os
os.getcwd()

'C:\\Users\\sunitha G\\dm_offshore\\projects\\fd_propensity\\output\\results'

In [74]:
log_to_file_algo(decile_results,output_path_results)

In [75]:
pd.crosstab(decile_df['decile'],decile_df['y_true'],margins=True)

y_true  0  1  All
decile           
1       0  1    1
2       1  0    1
All     1  1    2

In [76]:
df1 = pd.DataFrame({"A": np.random.randint(0,2,100), "B" : np.random.randint(0,2,100)})
pd.crosstab(df1.A, df1.B, margins=True)

B     0   1  All
A               
0    26  29   55
1    27  18   45
All  53  47  100